In [1]:
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
from collections import Counter
from tqdm import tqdm
import sys
sys.path.append("../../imports/")
import saver as sv

In [ ]:
wn.synsets('dog')

In [ ]:
wn.synsets('dog')[0].hyponyms()

In [ ]:
vocab = list(wn.all_synsets())
len(vocab)

In [ ]:
vocab_ = set()
for ss in vocab:
    word = ss.name().split('.')[0]
    if '_' in word:
        continue
    vocab_.add(word)
len(vocab_)

In [ ]:
def get_uniques(input_list):
    uniques = set()
    for k,v in Counter(input_list).items():
        if v>1:
            continue
        uniques.add(k)
    return uniques

In [ ]:
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
word_desc = dict()
for _word in tqdm(vocab_,position=0):
    word = lemmatizer.lemmatize(_word)
    synsets = wn.synsets(word)
    for synset in synsets:
        txt = synset.definition().lower()
        txt = re.sub(r'\W+', ' ', txt)
        words = [w for w in txt.split()]
        hypo = synset.hyponyms()
        hypo_words = [ss.name().split('.')[0] for ss in hypo]
        hyper = synset.hypernyms()
        hyper_words = [ss.name().split('.')[0] for ss in hyper]
        onto = hypo_words+hyper_words
        words = set([lemmatizer.lemmatize(w) for w in words if w not in stops])
        onto = set([lemmatizer.lemmatize(w) for w in onto if w not in stops])
        words = set(words)
        #add to dict
        if word in word_desc:
            word_desc[word].append((words,onto))
        else:
            word_desc[word]=[(words,onto)]

In [ ]:
sv.save(word_desc,"word_desc")

In [ ]:
word2desc = dict()
for k,v in word_desc.items():
    word2desc[k] = list()
    all_words = [item for sublist in v for item in sublist]
    #uniques = get_uniques(all_words)
    for words in v:
        words = set(words)
        #word2desc[k].append(words&uniques)
        word2desc[k].append(words)